In [1]:
from sklearn import ensemble, model_selection, datasets, metrics, tree, linear_model, preprocessing 
from sklearn.neural_network import MLPRegressor
import lightgbm as lgb
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
%matplotlib inline
from tqdm import tqdm

# Загрузка данных

In [2]:
!git clone https://github.com/CSSEGISandData/COVID-19 -b web-data COVID-19-new

Cloning into 'COVID-19-new'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 71881 (delta 12), reused 19 (delta 10), pack-reused 71853
Receiving objects: 100% (71881/71881), 599.79 MiB | 29.38 MiB/s, done.
Resolving deltas: 100% (47279/47279), done.


In [3]:
!git clone https://github.com/CSSEGISandData/COVID-19.git

Cloning into 'COVID-19'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 71881 (delta 12), reused 19 (delta 10), pack-reused 71853
Receiving objects: 100% (71881/71881), 600.93 MiB | 29.99 MiB/s, done.
Resolving deltas: 100% (47273/47273), done.
Checking out files: 100% (784/784), done.


In [4]:
!git clone https://github.com/grwlf/COVID-19_plus_Russia.git

Cloning into 'COVID-19_plus_Russia'...
remote: Enumerating objects: 27389, done.
remote: Total 27389 (delta 0), reused 0 (delta 0), pack-reused 27389
Receiving objects: 100% (27389/27389), 371.53 MiB | 32.10 MiB/s, done.
Resolving deltas: 100% (15667/15667), done.
Checking out files: 100% (708/708), done.


In [5]:
!git clone https://github.com/SamCullinan/Covid-19

Cloning into 'Covid-19'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (216/216), done.
remote: Total 222 (delta 126), reused 22 (delta 6), pack-reused 0
Receiving objects: 100% (222/222), 509.92 KiB | 6.45 MiB/s, done.
Resolving deltas: 100% (126/126), done.


# Подготовка данных

In [6]:
today = pd.to_datetime('today').strftime("%Y-%m-%d")

In [60]:
#Создание листа с датами с Июня до Ноября
today = pd.to_datetime('today').strftime("%Y-%m-%d")

dates_to_parse = []
for i in range(6, 12):
  for j in range(1, 32):
    if i < 10:
      if j < 10:
        date_to_append = '0' + str(i) + '-' + '0' + str(j) + '-2020'
      else:
        date_to_append = '0' + str(i) + '-' + str(j) + '-2020'
    else:
      if j < 10:
        date_to_append = str(i) + '-' + '0' + str(j) + '-2020'
      else:
        date_to_append = str(i) + '-' + str(j) + '-2020'
    dates_to_parse.append(date_to_append)

dates_to_remove = ['06-01-2020', '06-31-2020', '09-31-2020', '11-31-2020']

for date_to_remove in dates_to_remove:
  dates_to_parse.remove(date_to_remove)

#добавление в список дат за декабрь по сегодняшний день
for i in range(int(today[-2:])-1):
  if i < 10:
    dates_to_parse.append('12-0' + str(i+1) + '-2020')
  else:
    dates_to_parse.append('12-' + str(i+1) + '-2020')

In [61]:
#Сбор информации о зараженных, вылечившихся,умерших по датам с 83 регионов РФ с 01.06
data = pd.read_csv('/content/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/06-01-2020.csv')
data = data[data['Country_Region'] == 'Russia']
data['date'] = ['06-01-2020'] * 83

for date_to_parse in tqdm(dates_to_parse):
  tmp = pd.read_csv('/content/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/' + date_to_parse + '.csv')
  tmp = tmp[tmp['Country_Region'] == 'Russia']
  tmp['date'] = [date_to_parse] * 83
  data = pd.concat((data, tmp), axis = 0)

data = data[['Province_State', 'Confirmed', 'Deaths', 'Recovered', 'date']]
data.columns = ['region', 'confirmed', 'deaths', 'recovered', 'date']
regions = list(data['region'].unique())

100%|██████████| 186/186 [00:04<00:00, 39.65it/s]


In [62]:
#Загружаем данные заражений по России до 01.06
data_Ru = pd.read_csv("/content/COVID-19_plus_Russia/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_RU.csv").groupby("Province_State").sum()
data_Ru = data_Ru.iloc[:,15:].stack().reset_index()
data_Ru.columns = ["countries","date","confirmed"]

#Загружаем данные смертей по России до 01.06
data_deaths_Ru = pd.read_csv("/content/COVID-19_plus_Russia/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_RU.csv").groupby("Province_State").sum()
data_deaths_Ru = data_deaths_Ru.iloc[:,15:].stack().reset_index()
data_deaths_Ru.columns = ["countries","date","deaths"]
data_Ru = data_Ru.merge(data_deaths_Ru, how = 'left', left_on=('countries','date'), right_on=('countries','date'))
data_Ru["recovered"] = 0

#собираем в одну табличку данные до 01.06 и с 01.06

dates = []

for date in data_Ru['date']:
  dates.append(str(date[:2]) + '-' + str(date[3:5]) + '-2020')

data_Ru['date'] = dates

for date in data_Ru['date'].unique():
  dates.append(str(date[:2]) + '-' + str(date[3:5]) + '-2020')

data_Ru = data_Ru[data_Ru['date'].isin(dates[63:124])]
data_Ru.columns = ['region', 'date', 'confirmed', 'deaths', 'recovered']
data_Ru = data_Ru[data_Ru['region'].isin(regions)]

data = pd.concat((data_Ru, data))
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [63]:
#Загружаем данные по регионам России
regions_info = pd.read_csv("/content/Covid-19/russia_regions.csv")
regions_info = regions_info[["iso_code","csse_province_state","csse_province_state2","population","population_urban"]]
regions_info.population_urban = regions_info.population_urban/regions_info.population
regions_info.rename(columns={"iso_code": "iso_alpha3","csse_province_state":"ccse_name","csse_province_state2":"ccse_name2","population_urban":"urban_pop_rate"},inplace=True)

regions_info_2 = pd.read_csv("/content/Covid-19/regions-info.csv", decimal=",")
regions_info_2 = regions_info_2[["Region_eng","Density_pop_sqkm","Area"]]
regions_info_2.rename(columns={"Region_eng": "ccse_name2","Area": "land_area","Density_pop_sqkm": "density"},inplace=True)
regions_info = regions_info.merge(regions_info_2, how = 'left', left_on='ccse_name2', right_on='ccse_name2')

In [64]:
#Заменяем названия регионов для того, чтобы во всех данных были одинаковые названия
regions_to_replace = ['Altai Krai', 
                      'Amur Oblast', 
                      'Arkhangelsk Oblast', 
                      'Astrakhan Oblast', 
                      'Belgorod Oblast', 
                      'Bryansk Oblast',
                      'Vladimir Oblast', 
                      'Volgograd Oblast', 
                      'Vologda Oblast', 
                      'Voronezh Oblast', 
                      'Moscow', 
                      'Saint Petersburg',
                      'Sevastopol', 
                      'Jewish Autonomous Okrug', 
                      'Zabaykalsky Krai',
                      'Ivanovo Oblast', 
                      'Irkutsk Oblast',
                      'Kaliningrad Oblast', 
                      'Kaluga Oblast', 
                      'Kamchatka Krai', 
                      'Kemerovo Oblast',
                      'Kirov Oblast',
                      'Kostroma Oblast', 
                      'Krasnodar Krai', 
                      'Krasnoyarsk Krai', 
                      'Kurgan Oblast', 
                      'Kursk Oblast',
                      'Leningrad Oblast', 
                      'Lipetsk Oblast',
                      'Magadan Oblast',
                      'Moscow Oblast',
                      'Murmansk Oblast',
                      'Nenets Autonomous Okrug',
                      'Nizhny Novgorod Oblast',
                      'Novgorod Oblast',
                      'Novosibirsk Oblast',
                      'Omsk Oblast',
                      'Orenburg Oblast',
                      'Orel Oblast',
                      'Penza Oblast',
                      'Perm Krai',
                      'Primorsky Krai',
                      'Pskov Oblast',
                      'Adygea Republic',
                      'Altai Republic',
                      'Bashkortostan Republic',
                      'Buryatia Republic',
                      'Dagestan Republic',
                      'Ingushetia Republic',
                      'Kabardino-Balkarian Republic',
                      'Kalmykia Republic',
                      'Karachay-Cherkess Republic',
                      'Karelia Republic',
                      'Komi Republic',
                      'Republic of Crimea',
                      'Mari El Republic',
                      'Mordovia Republic',
                      'Sakha (Yakutiya) Republic',
                      'North Ossetia - Alania Republic',
                      'Tatarstan Republic',
                      'Tyva Republic',
                      'Udmurt Republic',
                      'Khakassia Republic',
                      'Chechen Republic',
                      'Rostov Oblast',
                      'Ryazan Oblast',
                      'Samara Oblast',
                      'Saratov Oblast',
                      'Sakhalin Oblast',
                      'Sverdlovsk Oblast',
                      'Smolensk Oblast',
                      'Stavropol Krai',
                      'Tambov Oblast',
                      'Tver Oblast',
                      'Tomsk Oblast',
                      'Tula Oblast',
                      'Tyumen Oblast',
                      'Ulyanovsk Oblast',
                      'Khabarovsk Krai',
                      'Khanty-Mansi Autonomous Okrug',
                      'Chelyabinsk Oblast',
                      'Chuvashia Republic',
                      'Chukotka Autonomous Okrug',
                      'Yamalo-Nenets Autonomous Okrug',
                      'Yaroslavl Oblast']

regions_info['region'] = regions_to_replace

In [65]:
#Информация о регионах
regions_info = regions_info[['region', 'population', 'urban_pop_rate', 'density', 'land_area']]

In [66]:
#Создание копии данных
data2 = data.copy()

In [67]:
#Объединение данных с информацией о регионах
data2 = pd.merge(data2, regions_info, on = 'region')

In [68]:
# Изменение числа зараженных и летальных исходов относительно предыдущих дней недели - дельты 
# Изменение дельт числа зараженных и летальных исходов

log_labels = True

if log_labels:
  data2.confirmed = np.log10(data2.confirmed+1)
  data2.deaths = np.log10(data2.deaths+1)
else:
  data2.confirmed = data2.confirmed
  data2.deaths = data2.deaths

data2.sort_values(['region', 'date'], inplace = True)
data2.reset_index(inplace = True)
data2.drop('index', axis = 1, inplace = True)

old_con = data2["confirmed"].iloc[:-1]
old_con2 = data2["deaths"].iloc[:-1]
old_recovered = data2["recovered"].iloc[:-1]
data2 = data2.iloc[1:]
data2["pred_conf"] = old_con.values
data2["pred_deaths"] = old_con2.values
data2["pred_recovered"] = old_recovered.values

if log_labels:
  data2["delta_conf"] = data2["confirmed"] - data2["pred_conf"]
  data2["delta_deaths"] = data2["deaths"] - data2["pred_deaths"]
else:
  data2["delta_conf"] = data2["confirmed"] - data2["pred_conf"]
  data2["delta_deaths"] = data2["deaths"] - data2["pred_deaths"]  

data2 = data2.iloc[1:]
data2["delta_conf1"] = old_con.values[1:] - old_con.values[:-1]
data2["delta_deaths1"] = old_con2.values[1:] - old_con2.values[:-1]

data2 = data2.iloc[1:]
data2["delta_conf2"] = old_con.values[1:-1] - old_con.values[:-2]
data2["delta_deaths2"] = old_con2.values[1:-1] - old_con2.values[:-2]
data2["delta_delta_conf1"] = data2["delta_conf2"] - data2["delta_conf1"]
data2 = data2.iloc[1:]
data2["delta_conf3"] = old_con.values[1:-2] - old_con.values[:-3]
data2["delta_deaths3"] = old_con2.values[1:-2] - old_con2.values[:-3]
data2["delta_delta_conf2"] = data2["delta_conf3"] - data2["delta_conf2"]
data2 = data2.iloc[1:]
data2["delta_conf4"] = old_con.values[1:-3] - old_con.values[:-4]
data2["delta_deaths4"] = old_con2.values[1:-3] - old_con2.values[:-4]
data2["delta_delta_conf3"] = data2["delta_conf4"] - data2["delta_conf3"]
data2 = data2.iloc[1:]
data2["delta_conf5"] = old_con.values[1:-4] - old_con.values[:-5]
data2["delta_deaths5"] = old_con2.values[1:-4] - old_con2.values[:-5]
data2["delta_delta_conf4"] = data2["delta_conf5"] - data2["delta_conf4"]
data2 = data2.iloc[1:]
data2["delta_conf6"] = old_con.values[1:-5] - old_con.values[:-6]
data2["delta_deaths6"] = old_con2.values[1:-5] - old_con2.values[:-6]
data2["delta_delta_conf5"] = data2["delta_conf6"] - data2["delta_conf5"]
data2["delta_conf_sum"] = (data2["delta_conf1"] + data2["delta_conf2"] + data2["delta_conf3"] + data2["delta_conf4"] + data2["delta_conf5"] + data2["delta_conf6"]) /6
data2["delta_deaths_sum"] = (data2["delta_deaths1"] + data2["delta_deaths2"] + data2["delta_deaths3"] + data2["delta_deaths4"] + data2["delta_deaths5"] + data2["delta_deaths6"]) /6
data2["delta_delta_sum"] = (data2["delta_delta_conf1"] + data2["delta_delta_conf2"] + data2["delta_delta_conf3"] + data2["delta_delta_conf4"] + data2["delta_delta_conf5"]) /5
data2["delta_loss"] = 0
data2.loc[data2["delta_conf1"] < data2["delta_conf_sum"],"delta_loss"] = 1
          
delta_data = data2["confirmed"].iloc[1:]

data2 = data2.iloc[:-1]
data2["delta2"] =  delta_data.values - data2["confirmed"] + data2["delta_conf"]
data2 = data2.iloc[:-1]
data2["delta3"] =  delta_data.values[1:]  - data2["confirmed"] + data2["delta_conf"]
data2 = data2.iloc[:-1]
data2["delta4"] = delta_data.values[2:] - data2["confirmed"] + data2["delta_conf"]
data2 = data2.iloc[:-1]
data2["delta5"] =  delta_data.values[3:] - data2["confirmed"] + data2["delta_conf"]
data2 = data2.iloc[:-1]
data2["delta6"] =  delta_data.values[4:] - data2["confirmed"] + data2["delta_conf"]
data2 = data2.iloc[:-1]
data2["delta7"] =  delta_data.values[5:] - data2["confirmed"] + data2["delta_conf"]
data2["delta_conf_count"] = 0
data2.loc[(data2["delta_conf1"] < data2["delta_conf2"]) | ((data2["delta_conf1"] <0.02) & (data2["delta_conf1"]>0)),"delta_conf_count"] +=0.2
data2.loc[(data2["delta_conf2"] < data2["delta_conf3"]) | ((data2["delta_conf2"] <0.02) & (data2["delta_conf2"]>0)),"delta_conf_count"] +=0.2
data2.loc[(data2["delta_conf3"] < data2["delta_conf4"]) | ((data2["delta_conf3"] <0.02) & (data2["delta_conf3"]>0)),"delta_conf_count"] +=0.2
data2.loc[(data2["delta_conf4"] < data2["delta_conf5"]) | ((data2["delta_conf4"] <0.02) & (data2["delta_conf4"]>0)),"delta_conf_count"] +=0.2
data2.loc[(data2["delta_conf5"] < data2["delta_conf6"]) | ((data2["delta_conf5"] <0.02) & (data2["delta_conf5"]>0)),"delta_conf_count"] +=0.2

data2['population_log'] = np.log10(data2.population+1)
data2['pred_conf-population_log'] = data2['pred_conf'] - data2['population_log'] + 3

In [69]:
data2.date = pd.to_datetime(data2.date)

In [70]:
#День появления i-ого кейса


end_date = "2020-05-01"
data4 = data2[["region","date"]].groupby("region").min()
data4.columns = ["Date_min"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2["days"] = (data2.date - data2.Date_min).dt.days

data4 = data2.loc[data2.confirmed > 0.5, ["region","date"]].groupby("region").min()
data4.columns = ["Date_5"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_5'] = data2['Date_5'].fillna(pd.to_datetime(end_date))
data2["days_5"] = (data2.date - data2.Date_5).dt.days


data4 = data2.loc[data2.confirmed > 0.8, ["region","date"]].groupby("region").min()
data4.columns = ["Date_8"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_8'] = data2['Date_8'].fillna(pd.to_datetime(end_date))
data2["days_8"] = (data2.date - data2.Date_8).dt.days


data4 = data2.loc[data2.confirmed > 1, ["region","date"]].groupby("region").min()
data4.columns = ["Date_10"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_10'] = data2['Date_10'].fillna(pd.to_datetime(end_date))
data2["days_10"] = (data2.date - data2.Date_10).dt.days


data4 = data2.loc[data2.confirmed > 1.5, ["region","date"]].groupby("region").min()
data4.columns = ["Date_50"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_50'] = data2['Date_50'].fillna(pd.to_datetime(end_date))
data2["days_50"] = (data2.date - data2.Date_50).dt.days


data4 = data2.loc[data2.confirmed > 1.8, ["region","date"]].groupby("region").min()
data4.columns = ["Date_80"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_80'] = data2['Date_80'].fillna(pd.to_datetime(end_date))
data2["days_80"] = (data2.date - data2.Date_80).dt.days


data4 = data2.loc[data2.confirmed > 2, ["region","date"]].groupby("region").min()
data4.columns = ["Date_100"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_100'] = data2['Date_100'].fillna(pd.to_datetime(end_date))
data2["days_100"] = (data2.date - data2.Date_100).dt.days


data4 = data2.loc[data2.confirmed > 2.5, ["region","date"]].groupby("region").min()
data4.columns = ["Date_500"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_500'] = data2['Date_500'].fillna(pd.to_datetime(end_date))
data2["days_500"] = (data2.date - data2.Date_500).dt.days


data4 = data2.loc[data2.confirmed > 3, ["region","date"]].groupby("region").min()
data4.columns = ["Date_1000"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_1000'] = data2['Date_1000'].fillna(pd.to_datetime(end_date))
data2["days_1000"] = (data2.date - data2.Date_1000).dt.days


data4 = data2.loc[data2.confirmed > 3.7, ["region","date"]].groupby("region").min()
data4.columns = ["Date_5000"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_5000'] = data2['Date_5000'].fillna(pd.to_datetime(end_date))
data2["days_5000"] = (data2.date - data2.Date_5000).dt.days

data4 = data2.loc[data2.confirmed > 4, ["region","date"]].groupby("region").min()
data4.columns = ["Date_10000"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_10000'] = data2['Date_10000'].fillna(pd.to_datetime(end_date))
data2["days_10000"] = (data2.date - data2.Date_10000).dt.days

data4 = data2.loc[data2.confirmed > 4.7, ["region","date"]].groupby("region").min()
data4.columns = ["Date_50000"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_50000'] = data2['Date_50000'].fillna(pd.to_datetime(end_date))
data2["days_50000"] = (data2.date - data2.Date_50000).dt.days

data4 = data2.loc[data2.confirmed > 5, ["region","date"]].groupby("region").min()
data4.columns = ["Date_100000"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_100000'] = data2['Date_100000'].fillna(pd.to_datetime(end_date))
data2["days_100000"] = (data2.date - data2.Date_100000).dt.days

data4 = data2.loc[data2.confirmed > 5, ["region","date"]].groupby("region").min()
data4.columns = ["Date_500000"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_500000'] = data2['Date_500000'].fillna(pd.to_datetime(end_date))
data2["days_500000"] = (data2.date - data2.Date_500000).dt.days

data2["days_mart"] = (data2.date - pd.to_datetime("2020-03-1")).dt.days


data4 = data2.loc[data2['pred_conf-population_log'] > 0, ["region","date"]].groupby("region").min()
data4.columns = ["Date_1_1000"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_1_1000'] = data2['Date_1_1000'].fillna(pd.to_datetime(end_date))
data2["days_1_1000"] = (data2.date - data2["Date_1_1000"]).dt.days
data2.loc[data2["days_1_1000"]<0,"days_1_1000"] = 0

data4 = data2.loc[data2['pred_conf-population_log'] > -1, ["region","date"]].groupby("region").min()
data4.columns = ["Date_1_100"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_1_100'] = data2['Date_1_100'].fillna(pd.to_datetime(end_date))
data2["days_1_100"] = (data2.date - data2["Date_1_100"]).dt.days


data4 = data2.loc[data2['pred_conf-population_log'] > -2, ["region","date"]].groupby("region").min()
data4.columns = ["Date_1_10000"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_1_10000'] = data2['Date_1_10000'].fillna(pd.to_datetime(end_date))
data2["days_1_10000"] = (data2.date - data2["Date_1_10000"]).dt.days


data4 = data2.loc[data2['pred_conf-population_log'] > -3, ["region","date"]].groupby("region").min()
data4.columns = ["Date_1_100000"]
data2 = data2.merge(data4, how = 'left', left_on='region', right_on='region')
data2['Date_1_100000'] = data2['Date_1_100000'].fillna(pd.to_datetime(end_date))
data2["days_1_100000"] = (data2.date - data2["Date_1_100000"]).dt.days


data2.drop(['Date_min','Date_10','Date_100','Date_1000', 'Date_5','Date_50','Date_500', 'Date_1_1000', 'Date_1_100', 'Date_1_10000'], axis='columns',inplace=True)
data2.date = data2["date"].apply(lambda x: pd.Series(x.strftime("%m-%d")))

In [71]:
# Устраняем проблему неустойчивости моделей к  большим значениям 

Scale = True

if Scale:
  data2.density = data2.density/data2.density.max()
  data2.land_area = np.log10(data2.land_area+1)
  data2.land_area = data2.land_area/data2.land_area.max()
  data2.population = data2.population/data2.population.max()
  data2.days = data2.days/100
  data2.days_10 = data2.days_10/100
  data2.days_100 = data2.days_100/100
  data2.days_1000 = data2.days_1000/100
  data2.days_10000 = data2.days_10000/100
  data2.days_100000 = data2.days_100000/100
  data2.days_5 = data2.days_5/100
  data2.days_50 = data2.days_50/100
  data2.days_500 = data2.days_500/100
  data2.days_5000 = data2.days_5000/100
  data2.days_50000 = data2.days_50000/100
  data2.days_500000 = data2.days_500000/100
  data2.days_8 = data2.days_8/100
  data2.days_80 = data2.days_80/100    
  data2.days_1_100 = data2.days_1_100/100  
  data2.days_1_1000 = data2.days_1_1000/100 
  data2.days_1_10000 = data2.days_1_10000/100
  data2.days_1_100000 = data2.days_1_100000/100 
  data2.days_mart = data2.days_mart/100

  sc_days = 100
else:
  sc_days = 1

In [72]:
dates = list(data2['date'].unique())

In [73]:
data2 = data2[data2['date'].isin(dates[:-7])].copy()

In [74]:
data2

,region,date,confirmed,deaths,recovered,population,urban_pop_rate,density,land_area,pred_conf,pred_deaths,pred_recovered,delta_conf,delta_deaths,delta_conf1,delta_deaths1,delta_conf2,delta_deaths2,delta_delta_conf1,delta_conf3,delta_deaths3,delta_delta_conf2,delta_conf4,delta_deaths4,delta_delta_conf3,delta_conf5,delta_deaths5,delta_delta_conf4,delta_conf6,delta_deaths6,delta_delta_conf5,delta_conf_sum,delta_deaths_sum,delta_delta_sum,delta_loss,delta2,delta3,delta4,delta5,delta6,delta7,delta_conf_count,population_log,pred_conf-population_log,days,days_5,Date_8,days_8,days_10,days_50,Date_80,days_80,days_100,days_500,days_1000,Date_5000,days_5000,Date_10000,days_10000,Date_50000,days_50000,Date_100000,days_100000,Date_500000,days_500000,days_mart,days_1_1000,days_1_100,days_1_10000,Date_1_100000,days_1_100000
0,Adygea Republic,04-08,1.146128,0.000000,0.0,0.036514,0.472835,0.012001,0.599734,1.146128,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.243038,0.000000,0.243038,0.000000,0.000000,-0.243038,0.124939,0.000000,0.124939,0.000000,0.000000,-0.124939,0.079181,0.000000,0.079181,0.074526,0.000000,0.015836,1,0.000000,0.176091,0.176091,0.176091,0.176091,0.301030,0.6,5.666007,-1.519879,0.00,0.00,2020-04-08,0.00,0.00,-0.08,2020-04-17,-0.09,-0.12,-0.38,-0.60,2020-10-21,-1.96,2020-05-01,-0.23,2020-05-01,-0.23,2020-05-01,-0.23,2020-05-01,-0.23,0.38,0.00,-0.10,0.00,2020-04-08,0.00
1,Adygea Republic,04-09,1.146128,0.000000,0.0,0.036514,0.472835,0.012001,0.599734,1.146128,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.243038,0.000000,0.243038,0.000000,0.000000,-0.243038,0.124939,0.000000,0.124939,0.000000,0.000000,-0.124939,0.061329,0.000000,0.000000,1,0.176091,0.176091,0.176091,0.176091,0.301030,0.301030,0.4,5.666007,-1.519879,0.01,0.01,2020-04-08,0.01,0.01,-0.07,2020-04-17,-0.08,-0.11,-0.37,-0.59,2020-10-21,-1.95,2020-05-01,-0.22,2020-05-01,-0.22,2020-05-01,-0.22,2020-05-01,-0.22,0.39,0.00,-0.09,0.01,2020-04-08,0.01
2,Adygea Republic,04-10,1.322219,0.000000,0.0,0.036514,0.472835,0.012001,0.599734,1.146128,0.000000,0.0,0.176091,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.243038,0.000000,0.243038,0.000000,0.000000,-0.243038,0.124939,0.000000,0.124939,0.061329,0.000000,0.024988,1,0.176091,0.176091,0.176091,0.301030,0.301030,0.497325,0.4,5.666007,-1.519879,0.02,0.02,2020-04-08,0.02,0.02,-0.06,2020-04-17,-0.07,-0.10,-0.36,-0.58,2020-10-21,-1.94,2020-05-01,-0.21,2020-05-01,-0.21,2020-05-01,-0.21,2020-05-01,-0.21,0.40,0.00,-0.08,0.02,2020-04-08,0.02
3,Adygea Republic,04-11,1.322219,0.000000,0.0,0.036514,0.472835,0.012001,0.599734,1.322219,0.000000,0.0,0.000000,0.000000,0.176091,0.000000,0.000000,0.000000,-0.176091,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.243038,0.000000,0.243038,0.000000,0.000000,-0.243038,0.069855,0.000000,-0.035218,0,0.000000,0.000000,0.124939,0.124939,0.321233,0.580871,0.2,5.666007,-1.343787,0.03,0.03,2020-04-08,0.03,0.03,-0.05,2020-04-17,-0.06,-0.09,-0.35,-0.57,2020-10-21,-1.93,2020-05-01,-0.20,2020-05-01,-0.20,2020-05-01,-0.20,2020-05-01,-0.20,0.41,0.00,-0.07,0.03,2020-04-08,0.03
4,Adygea Republic,04-12,1.322219,0.000000,0.0,0.036514,0.472835,0.012001,0.599734,1.322219,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.176091,0.000000,0.176091,0.000000,0.000000,-0.176091,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.243038,0.000000,0.243038,0.069855,0.000000,0.048608,1,0.000000,0.124939,0.124939,0.321233,0.580871,0.607200,0.4,5.666007,-1.343787,0.04,0.04,2020-04-08,0.04,0.04,-0.04,2020-04-17,-0.05,-0.08,-0.34,-0.56,2020-10-21,-1.92,2020-05-01,-0.19,2020-05-01,-0.19,2020-05-01,-0.19,2020-05-01,-0.19,0.42,0.00,-0.06,0.04,2020-04-08,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20566,Z

In [79]:
# Предсказание числа зараженных 
def prog_delta_log(model):
  prediction_confirmed = model.predict(new_data_list)
  if log_labels:
    prediction_confirmed = prediction_confirmed  + new_data_list.pred_conf.values
    prediction_confirmed = 10**prediction_confirmed - 1
  else:
    prediction_confirmed = prediction_confirmed + np.log10(abs(new_data_list.delta_conf1.values)+1)
    prediction_confirmed = 10**prediction_confirmed - 1 + new_data_list.pred_conf.values
    prediction_confirmed = prediction_confirmed 
  return prediction_confirmed


# Функция построения графиков 
def plot_new(list_regions, Matrix, subplot_x = 2, days_before = 10, point = True):
  dat2 = data2.copy()
  plt.rcParams['figure.figsize'] = [20, len(list_regions)*20/subplot_x**2]
  if log_labels:
    dat2.confirmed = 10**dat2.confirmed - 1
  else:
    dat2.confirmed = dat2.confirmed
  for i, Region in enumerate(list_regions):
    plt.subplot(len(list_regions)// subplot_x + 1, subplot_x, i+1)
    yyy = dat2.loc[(dat2.region == Region)&(dat2.confirmed>0)&(dat2.days_mart>=(days_x - days_before)/sc_days)]

    if point:
      plt.plot(yyy.date, yyy.confirmed, label = "Истинное значение", marker='o')
      ax = plt.gca()
      for x,y,i in zip(yyy.date,yyy.confirmed,range(0,len(yyy))):
        if not i % ((subplot_x * (days_before+days_prog))//20):
          plt.annotate(int(y), (x,y), textcoords="offset points", xytext=(-5,10), ha="right", va = "bottom")
    else:
      plt.plot(yyy.date, yyy.confirmed, label = "Истинное значение")
    plt.gca().set(xlabel='Дата', ylabel='Заражения')
    yyy2 = Matrix.loc[:,[Region]]
    if point:
      plt.plot(yyy2, label = "name", marker='o')
      ax = plt.gca()
      for x,y,i in zip(yyy2.index,yyy2.values.T[0],range(0,len(yyy2))):
        if not i % ((subplot_x * (days_before+days_prog))//20):
          plt.annotate(int(y), (x,y), textcoords="offset points", xytext=(5,-10), ha="left", va = "top")
    else:
      plt.plot(yyy2, label = "name")
    plt.title(Region)
    plt.xticks(np.arange(0, (days_before+days_prog), ((days_before+days_prog)//10)*subplot_x))
    plt.grid(True)
    plt.legend()  

# Функция ошибки MALE 
def pred_score(predictions, day):
  yyy = new_data[(new_data.days_mart==(days_x + day - 1)/sc_days) & (new_data.confirmed > 0)]
  region_list_x = new_data[(new_data.days_mart==(days_x)/sc_days) & (new_data.confirmed > 0)].region.to_list()
  yyy = yyy[yyy.region.isin(region_list_x)]
  if len(yyy)==len(predictions):
    if log_labels:
      labels = 10**yyy.confirmed.values  - 1
    else:
      labels = yyy.confirmed.values
    return np.mean(np.abs(np.log10((predictions+1)/(labels+1)))) 
  else:
    return 0

# Конфигурация модели

In [80]:
#Выберите день начала приватных данных начиная с 1 марта
days_x =  272
iterat = 1000
days_prog = 8

#type_ans_delta - Предсказывать общее число, или изменение за день
type_ans_delta = True

In [81]:
# Конфигурация модели 
param_list =['region','confirmed', 'deaths', 'delta_conf', 'delta_deaths',
              'pred_conf', 'pred_deaths', 'delta_conf1', 'delta_deaths1',
             'delta2',  'delta3',  'delta4',  'delta5',  'delta6',  'delta7', 
              'delta_conf_count',
             'delta_deaths2','delta_deaths3','delta_deaths4','delta_deaths5','delta_deaths6',
             'urban_pop_rate',             
              'pred_conf-population_log',              
             'population_log',
             'land_area',
             'days',
             'days_10',
             'days_100',
             'days_5',
             'days_50',
             'days_500',
             'days_8',
             'days_80',                        
             'days_1_100', 
             'days_1_1000',              
             'days_1_10000',
             'delta_loss',  
             'delta_delta_conf1',
             'delta_delta_conf2',
             'delta_delta_conf3',
             'delta_delta_conf4',
             'delta_delta_conf5',
             'delta_conf2', 
             'delta_conf3', 
             'delta_conf4', 
             'delta_conf5', 
             'delta_conf6', 
             'days_mart']

data3 = data2[param_list]

model_Confirmed, model_Death = {}, {}

new_data = data3[data3.days_mart >= (days_x)/sc_days]

data_x = new_data[new_data.days_mart == (days_x)/sc_days]
old_data = data3[(data3.days_mart <= (days_x-5)/sc_days) & (data3.confirmed > 0)]

if log_labels:
  train_labels = old_data.delta_conf
  delta2 = old_data.delta2
  delta3 = old_data.delta3
  delta4 = old_data.delta4
  delta5 = old_data.delta5
  delta6 = old_data.delta6
  delta7 = old_data.delta7
else:  
  train_labels = - np.log10(abs(old_data.delta_conf1)+1) + np.log10(abs(old_data.delta_conf)+1)
  delta2 = - np.log10(abs(old_data.delta_conf1)+1) + np.log10(abs(old_data.delta2)+1)
  delta3 = - np.log10(abs(old_data.delta_conf1)+1) + np.log10(abs(old_data.delta3)+1)
  delta4 = - np.log10(abs(old_data.delta_conf1)+1) + np.log10(abs(old_data.delta4)+1)
  delta5 = - np.log10(abs(old_data.delta_conf1)+1) + np.log10(abs(old_data.delta5)+1)
  delta6 = - np.log10(abs(old_data.delta_conf1)+1) + np.log10(abs(old_data.delta6)+1)
  delta7 = - np.log10(abs(old_data.delta_conf1)+1) + np.log10(abs(old_data.delta7)+1)

#Убираем ответ из данных
train_data = old_data.drop(['delta2',  'delta3',  'delta4',  'delta5',  'delta6',  'delta7', 'region','confirmed','deaths',"delta_conf","delta_deaths"], axis='columns')
new_data_list = data_x.drop(['delta2',  'delta3',  'delta4',  'delta5',  'delta6',  'delta7', 'region','confirmed','deaths',"delta_conf","delta_deaths"], axis='columns')  

,region,confirmed,deaths,delta_conf,delta_deaths,pred_conf,pred_deaths,delta_conf1,delta_deaths1,delta2,delta3,delta4,delta5,delta6,delta7,delta_conf_count,delta_deaths2,delta_deaths3,delta_deaths4,delta_deaths5,delta_deaths6,urban_pop_rate,pred_conf-population_log,population_log,land_area,days,days_10,days_100,days_5,days_50,days_500,days_8,days_80,days_1_100,days_1_1000,days_1_10000,delta_loss,delta_delta_conf1,delta_delta_conf2,delta_delta_conf3,delta_delta_conf4,delta_delta_conf5,delta_conf2,delta_conf3,delta_conf4,delta_conf5,delta_conf6,days_mart
0,Adygea Republic,1.146128,0.000000,0.000000,0.000000,1.146128,0.000000,0.000000,0.000000,0.000000,0.176091,0.176091,0.176091,0.176091,0.301030,0.6,0.000000,0.000000,0.000000,0.000000,0.000000,0.472835,-1.519879,5.666007,0.599734,0.00,0.00,-0.12,0.00,-0.08,-0.38,0.00,-0.09,-0.10,0.00,0.00,1,0.243038,-0.243038,0.124939,-0.124939,0.079181,0.243038,0.000000,0.124939,0.000000,0.079181,0.38
1,Adygea Republic,1.146128,0.000000,0.000000,0.000000,1.146128,0.000000,0.000000,0.000000,0.176091,0.176091,0.176091,0.176091,0.301030,0.301030,0.4,0.000000,0.000000,0.000000,0.000000,0.000000,0.472835,-1.519879,5.666007,0.599734,0.01,0.01,-0.11,0.01,-0.07,-0.37,0.01,-0.08,-0.09,0.00,0.01,1,0.000000,0.243038,-0.243038,0.124939,-0.124939,0.000000,0.243038,0.000000,0.124939,0.000000,0.39
2,Adygea Republic,1.322219,0.000000,0.176091,0.000000,1.146128,0.000000,0.000000,0.000000,0.176091,0.176091,0.176091,0.301030,0.301030,0.497325,0.4,0.000000,0.000000,0.000000,0.000000,0.000000,0.472835,-1.519879,5.666007,0.599734,0.02,0.02,-0.10,0.02,-0.06,-0.36,0.02,-0.07,-0.08,0.00,0.02,1,0.000000,0.000000,0.243038,-0.243038,0.124939,0.000000,0.000000,0.243038,0.000000,0.124939,0.40
3,Adygea Republic,1.322219,0.000000,0.000000,0.000000,1.322219,0.000000,0.176091,0.000000,0.000000,0.000000,0.124939,0.124939,0.321233,0.580871,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.472835,-1.343787,5.666007,0.599734,0.03,0.03,-0.09,0.03,-0.05,-0.35,0.03,-0.06,-0.07,0.00,0.03,0,-0.176091,0.000000,0.000000,0.243038,-0.243038,0.000000,0.000000,0.000000,0.243038,0.000000,0.41
4,Adygea Republic,1.322219,0.000000,0.000000,0.000000,1.322219,0.000000,0.000000,0.000000,0.000000,0.124939,0.124939,0.321233,0.580871,0.607200,0.4,0.000000,0.000000,0.000000,0.000000,0.000000,0.472835,-1.343787,5.666007,0.599734,0.04,0.04,-0.08,0.04,-0.04,-0.34,0.04,-0.05,-0.06,0.00,0.04,1,0.176091,-0.176091,0.000000,0.000000,0.243038,0.176091,0.000000,0.000000,0.000000,0.243038,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20561,Zabaykalsky Krai,4.211974,2.394452,0.007258,0.010636,4.204717,2.383815,0.007353,0.010903,0.014317,0.021367,0.028254,0.035058,0.041733,0.048233,1.0,0.014977,0.015512,0.009984,0.006103,0.006190,0.681972,1.179551,6.025166,0.868444,2.32,2.20,2.05,2.25,2.15,1.92,2.21,2.10,2.32,2.32,2.32,1,0.000043,0.000071,0.000044,0.000073,0.000105,0.007396,0.007467,0.007511,0.007584,0.007688,2.63
20562,Zabaykalsky Krai,4.219034,2.403121,0.007060,0.008669,4.211974,2.394452,0.007258,0.010636,0.014110,0.020996,0.027800,0.034475,0.040976,0.047428,1.0,0.010903,0.014977,0.015512,0.009984,0.006103,0.681972,1.186809,6.025166,0.868444,2.33,2.21,2.06,2.26,2.16,1.93,2.22,2.11,2.33,2.33,2.33,1,0.000096,0.000043,0.000071,0.000044,0.000073,0.007353,0.007396,0.007467,0.007511,0.007584,2.64
20563,Zabaykalsky Krai,4.226084,2.411620,0.007050,0.008499,4.219034,2.403121,0.007060,0.008669,0.013937,0.020741,0.027415,0.033916,0.040369,0.046609,1.0,0.010636,0.010903,0.014977,0.015512,0.009984,0.681972,1.193868,6.025166,0.868444,2.34,2.22,2.07,2.27,2.17,1.94,2.23,2.12,2.34,2.34,2.34,1,0.000198,0.000096,0.000043,0.000071,0.000044,0.007258,0.007353,0.007396,0.007467,0.007511,2.65
20564,Zabaykalsky Krai,4.232971,2.428135,0.006887,0.016515,4.226084,2.411620,0.007050,0.008499,0.013691,0.020365,0.026866,0.033319,0.039559,0.04

# Модели

Обучаем lightgbm

In [82]:
params = {} 
model_Confirmed["lightgbm"] = []
d_train = lgb.Dataset(train_data, label=train_labels)
model_Confirmed["lightgbm"].append(lgb.train(params, d_train, iterat))
d_train = lgb.Dataset(train_data[delta2 >= 0], delta2[delta2 >= 0])
model_Confirmed["lightgbm"].append(lgb.train(params, d_train, iterat))
d_train = lgb.Dataset(train_data[delta3 >= 0], delta3[delta3 >= 0])
model_Confirmed["lightgbm"].append(lgb.train(params, d_train, iterat))
d_train = lgb.Dataset(train_data[delta4 >= 0], delta4[delta4 >= 0])
model_Confirmed["lightgbm"].append(lgb.train(params, d_train, iterat))
d_train = lgb.Dataset(train_data[delta5 >= 0], delta5[delta5 >= 0])
model_Confirmed["lightgbm"].append(lgb.train(params, d_train, iterat))
d_train = lgb.Dataset(train_data[delta6 >= 0], delta6[delta6 >= 0])
model_Confirmed["lightgbm"].append(lgb.train(params, d_train, iterat))
d_train = lgb.Dataset(train_data[delta7 >= 0], delta7[delta7 >= 0])
model_Confirmed["lightgbm"].append(lgb.train(params, d_train, iterat))

Обучаем Xgboost

In [87]:
model_Confirmed["Xgboost"] = []
model_Confirmed["Xgboost"].append(xgb.XGBRegressor(objective ='reg:squarederror',n_estimators=iterat, max_depth=30, 
                                              n_jobs = -1).fit(train_data, train_labels))
model_Confirmed["Xgboost"].append(xgb.XGBRegressor(objective ='reg:squarederror',n_estimators=iterat, max_depth=30, 
                                              n_jobs = -1).fit(train_data[delta2 >= 0], delta2[delta2 >= 0]))
model_Confirmed["Xgboost"].append(xgb.XGBRegressor(objective ='reg:squarederror',n_estimators=iterat, max_depth=30, 
                                              n_jobs = -1).fit(train_data[delta3 >= 0], delta3[delta3 >= 0]))
model_Confirmed["Xgboost"].append(xgb.XGBRegressor(objective ='reg:squarederror',n_estimators=iterat, max_depth=30, 
                                              n_jobs = -1).fit(train_data[delta4 >= 0], delta4[delta4 >= 0]))
model_Confirmed["Xgboost"].append(xgb.XGBRegressor(objective ='reg:squarederror',n_estimators=iterat, max_depth=30, 
                                              n_jobs = -1).fit(train_data[delta5 >= 0], delta5[delta5 >= 0]))
model_Confirmed["Xgboost"].append(xgb.XGBRegressor(objective ='reg:squarederror',n_estimators=iterat, max_depth=30, 
                                              n_jobs = -1).fit(train_data[delta6 >= 0], delta6[delta6 >= 0]))
model_Confirmed["Xgboost"].append(xgb.XGBRegressor(objective ='reg:squarederror',n_estimators=iterat, max_depth=30, 
                                              n_jobs = -1).fit(train_data[delta7 >= 0], delta7[delta7 >= 0]))

Обучаем Adaboost

In [ ]:
treeDepth = 30
mdl = tree.DecisionTreeRegressor(max_depth=treeDepth)
param_grid = {
    'n_estimators': [iterat//10],
    'learning_rate': [0.0001],
    'loss' : ["exponential"]}
regrMdl = ensemble.AdaBoostRegressor(base_estimator=mdl)

model_Confirmed["Adaboost"] = []
model_Confirmed["Adaboost"].append(model_selection.GridSearchCV(estimator = regrMdl, param_grid = param_grid, 
                                         cv = 2, n_jobs = -1).fit(train_data,train_labels))
model_Confirmed["Adaboost"].append(model_selection.GridSearchCV(estimator = regrMdl, param_grid = param_grid, 
                                         cv = 2, n_jobs = -1).fit(train_data[delta2 >= 0], delta2[delta2 >= 0]))
model_Confirmed["Adaboost"].append(model_selection.GridSearchCV(estimator = regrMdl, param_grid = param_grid, 
                                         cv = 2, n_jobs = -1).fit(train_data[delta3 >= 0], delta3[delta3 >= 0]))
model_Confirmed["Adaboost"].append(model_selection.GridSearchCV(estimator = regrMdl, param_grid = param_grid, 
                                         cv = 2, n_jobs = -1).fit(train_data[delta4 >= 0], delta4[delta4 >= 0]))
model_Confirmed["Adaboost"].append(model_selection.GridSearchCV(estimator = regrMdl, param_grid = param_grid, 
                                         cv = 2, n_jobs = -1).fit(train_data[delta5 >= 0], delta5[delta5 >= 0]))
model_Confirmed["Adaboost"].append(model_selection.GridSearchCV(estimator = regrMdl, param_grid = param_grid, 
                                         cv = 2, n_jobs = -1).fit(train_data[delta6 >= 0], delta6[delta6 >= 0]))
model_Confirmed["Adaboost"].append(model_selection.GridSearchCV(estimator = regrMdl, param_grid = param_grid, 
                                         cv = 2, n_jobs = -1).fit(train_data[delta7 >= 0], delta7[delta7 >= 0]))

# Предсказываем новые данные

In [ ]:
predictions_mean = np.zeros((8,len(new_data_list)))
for model in model_Confirmed:
  if log_labels:
    predictions = [10**new_data_list.pred_conf.values-1]
  else:
    predictions = [new_data_list.pred_conf.values]
  for mod in model_Confirmed[model]:
    pred = prog_delta_log(mod)
    pred[pred < predictions[len(predictions)-1]] = predictions[len(predictions)-1][pred < predictions[len(predictions)-1]] 
    predictions.append(pred)
  predictions_mean += np.array(predictions)
  scores = []
  for i,pred in enumerate(predictions):
      score = pred_score(pred,i)
      if score:
        scores.append(score)
  print(model, np.sum(scores), scores)      

predictions_mean = predictions_mean/len(model_Confirmed)
scores = []
for i,pred in enumerate(predictions_mean):
    score = pred_score(pred,i)
    if score:
      scores.append(score)
print("Среднее", np.sum(scores), scores)

labels = data_x.region
data_list = pd.date_range('2020-11-27', periods = 7+1, freq ='d')
data_list = data_list.strftime('%#m-%#d') 
Matrix = pd.DataFrame(predictions_mean,columns = labels,index = data_list)

# Создаём список регионов
list_regions = ['Moscow', 'Moscow Oblast', 'Saint Petersburg', 'Chechen Republic', 'Adygea Republic', 'Dagestan Republic']
# Количество графиков в строке
subplot_x = 2

# Отмечать точки
point = True

#За сколько дней до начала прогноза
days_before = 30

plot_new(list_regions, Matrix, subplot_x, days_before, point)